In [8]:
# 引入套件
import pandas as pd
import numpy as np

In [ ]:
import openpyxl
df = pd.read_excel("../文章原始資料/0926_new.xlsx")

In [10]:
df

,title,tag,content,url
0,在狂牛症之後，英國如何從食安危機裡覺醒？,社會群體 醫療健康 科學傳播 文明足跡 環境生態,面對食安危機，政府的應對措施竟是舉辦烹飪比賽，或是讓高官在鏡頭前大啖美食。這不是台灣，而是被...,https://pansci.asia/archives/98560
1,歐洲酸沼特產，木乃伊「圖論男子」的那時此刻——《木乃伊不容易》,環境生態 自然生態 好書搶先看 植物王國 化學物語 人體解析 生命奧祕 活得科學 文明足跡 ...,作者／李衍蒨「圖論男子」木乃伊檔案發現地點：丹麥數目：1 具發現過程：於沼澤範圍被找到特點：...,https://pansci.asia/archives/170550
2,「抗體檢驗」是什麼？面臨那些限制？偽陽性、抽樣與群體免疫,來自台灣 活得科學 醫療健康 社會群體,抗體檢驗是什麼？人體感染病原體後一段時間，免疫系統會製作針對病原體的抗體，即使痊癒後，抗體仍...,https://pansci.asia/archives/189659
3,在布達佩斯沿著多瑙河畔，來一趟數學朝聖之旅吧！,數學妙用 文明足跡,文／陳宏賓 │ UniMath 主編、逢甲大學應用數學系助理教授。多瑙河夜景：前方為伊莉莎白...,https://pansci.asia/archives/138379
4,2500年的木乃伊重見天日：少年米尼爾迪斯,文明足跡 科學傳播 透視科學 泛科授權 1.0 泛科授權 2.0 生命奧祕,米尼爾迪斯和他的陪葬面具。Photograph: Charles Rex Arbogast/...,https://pansci.asia/archives/72532
...,...,...,...,...
9812,太空:最終回收目標?,科技能源,作者：Roger East（Green Futures）對太空計畫而言，回收與再利用概念還很...,https://pansci.asia/archives/14096
9813,用App Inventor直接控制樂高EV3機器人：單點控制,科技能源,文／CAVEDU教育團隊各位讀者，2015 快樂！ CAVEDU的專欄從2012開始連載， ...,https://pansci.asia/archives/75189
9814,輕鬆使用Android裝置控制樂高機器人：按鈕控制,科技能源,文／曾吉弘（CAVE教育團隊http://www.cavedu.com）用手機遙控機器人，聽...,https://pansci.asia/archives/12611
9815,這裡不是片場 — 全世界最大的救災實境演練場地「守護者中心」,活得科學,文 / 陳妤寧Guardian Centers 的模擬救災現場（圖片來源：Guardian ...,https://pansci.asia/archives/62191


In [11]:
# Import modules
from sentence_transformers import SentenceTransformer, util
# Select model by transformer
# about model: https://huggingface.co/sentence-transformers/paraphrase-multilingual-mpnet-base-v2
sbert_model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')
print('sbert_model loaded')
def embedding_sentence(sentence):
    try:
        return {
            "state" : True,
            "value" : sbert_model.encode(sentence).tolist(), # return a vector list
        }
    except Exception as exp:
        return {
            "state" : False,
            "value" : np.zeros(768)
        }


c:\Users\trello\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


sbert_model loaded


In [13]:
# 假设你的DataFrame为df
df['vector_title'] = df['title'].apply(lambda x: embedding_sentence(x)['value'])

In [14]:
df['vector_content'] = df['content'].apply(lambda x: embedding_sentence(x)['value'])

In [15]:
df.to_json("../backup/pansci_vec.json")

In [30]:
df.to_json("pansci_vec.json")